In [56]:
import pandas as pd

In [57]:
df_movies = pd.read_csv('ml-latest/movies.csv')

In [58]:
df_ratings = pd.read_csv('ml-latest/ratings.csv')
df_ratings.set_index('movieId', inplace=True)

In [59]:
df_ratings["no_people_rated"] = [1 for _ in range(len(df_ratings))]

In [60]:
grouped_ratings = df_ratings.groupby('movieId')

grouped_ratings = grouped_ratings.sum()


In [61]:
grouped_ratings

,userId,rating,timestamp,no_people_rated
movieId,,,,
1,12696689201,299072.0,90676232180619,76813
2,4969411174,99030.5,34965684106628,30209
3,2608079724,50169.5,15487105413879,15820
4,502841790,8685.5,2874569542209,3028
5,2603834260,48619.0,15955867010862,15801
...,...,...,...,...
288967,47791,3.5,1689748357,1
288971,98408,0.5,1689798322,1
288975,154483,4.0,1689812351,1


In [62]:
grouped_ratings["avg_rating"] = grouped_ratings["rating"]/grouped_ratings["no_people_rated"]

In [63]:
grouped_ratings.sort_values('avg_rating', ascending=False, inplace=True)
grouped_ratings = grouped_ratings[grouped_ratings["no_people_rated"]>1000]
grouped_ratings

,userId,rating,timestamp,no_people_rated,avg_rating
movieId,,,,,
171011,343910470,9086.0,3186768141626,2041,4.451739
159817,503560410,13411.0,4681464629694,3015,4.448093
170705,470416403,12542.0,4464049315381,2835,4.423986
318,20213155335,540156.0,156345852293172,122296,4.416792
202439,2071618320,53687.0,20015714719947,12399,4.329946
...,...,...,...,...,...
2555,285063817,2896.5,1898360097339,1679,1.725134
57532,194063825,1903.0,1636766416042,1172,1.623720
43919,183721607,1828.0,1496965366895,1130,1.617699


In [64]:
df_movies = pd.read_csv('ml-latest/movies.csv')
pop_genres = ["Action", "Adventure", "Animation", "Comedy", "Crime", "Drama", 
              "Fantasy", "Horror", "Mystery", "Romance", "Sci-Fi", "Thriller"]
hm_suitability = {}
genre_suitable_lst = []
for i in range(len(df_movies)):
    genre_temp_storage = ""
    movie = df_movies.iloc[i]
    all_genres = movie["genres"]
    is_suitable = 0
    for genre in pop_genres: 
        if genre in all_genres:
            is_suitable += 1
            genre_temp_storage = genre
    if is_suitable == 1:
        hm_suitability[movie["movieId"]] = genre_temp_storage
    else:
        hm_suitability[movie["movieId"]] = "Not suitable"
df_suitability = pd.DataFrame()
df_suitability["movieId"] = hm_suitability.keys()
df_suitability["suitable_genre"] = hm_suitability.values()
df_suitability.set_index("movieId", inplace=True)
df_suitability = df_suitability[df_suitability["suitable_genre"] != "Not suitable"]
df_suitability

,suitable_genre
movieId,
5,Comedy
8,Adventure
9,Action
14,Drama
18,Comedy
...,...
288953,Comedy
288955,Drama
288957,Horror


join using the movie id

In [65]:
grouped_ratings = grouped_ratings.join(df_suitability, how="inner")
grouped_ratings

,userId,rating,timestamp,no_people_rated,avg_rating,suitable_genre
movieId,,,,,,
1203,3738563090,96992.5,29894679700250,22730,4.267158,Drama
527,13929219732,357340.5,103671567916204,84232,4.242337,Drama
1193,8118516317,207758.5,61568690477830,49316,4.212801,Drama
750,5650978895,144160.5,42473159316615,34324,4.199991,Comedy
26082,213092748,5374.0,1897779020524,1282,4.191888,Drama
...,...,...,...,...,...,...
1981,243854131,2772.0,1714465547406,1495,1.854181,Horror
1760,552474506,6083.5,3830780656494,3317,1.834037,Comedy
6482,381293121,4065.5,2992591178695,2292,1.773778,Comedy
